In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import re
import random
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import random

from .utils import *
from .constants import *
from ObstacleDetectionModel import *
from Datasets import *


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
pgm_dir = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/data/obsta/pgm'
seg_dir = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/data/obsta/seg'

# Pattern for extracting timestamp from segmentation images
pattern = re.compile(r"(\d+)_(\d+\.?\d*)_(\d+\.?\d*)_(-?\d+\.?\d*)_(-?\d+\.?\d*)\.png$")

# Prepare a list of matching PGM and segmentation files based on timestamps
matched_files = []
for seg_file in os.listdir(seg_dir):
    if seg_file.endswith('.png'):
        match = pattern.match(seg_file)
        if match:
            timestamp = match.group(1)
            obstacle = int(match.group(2))  # Only using obstacle detection data
            pgm_file = f"{timestamp}.pgm"
            pgm_path = os.path.join(pgm_dir, pgm_file)
            seg_path = os.path.join(seg_dir, seg_file)
            if os.path.exists(pgm_path):
                matched_files.append((pgm_path, seg_path, obstacle))

obstacle_0_files = [item for item in matched_files if item[2] == 0]
obstacle_1_files = [item for item in matched_files if item[2] == 1]

# Determine the upsampling count to balance the classes
upsample_count = len(obstacle_0_files)  # Aim to match the count of obstacle 0 files

# Upsample obstacle 1 files
upsampled_obstacle_1_files = obstacle_1_files * (upsample_count // len(obstacle_1_files))  # Full duplications
remaining = upsample_count % len(obstacle_1_files)  # Remainder to reach target count
upsampled_obstacle_1_files += random.sample(obstacle_1_files, remaining)  # Randomly sample additional files if needed

# Combine upsampled obstacle 1 files with obstacle 0 files
balanced_files = obstacle_0_files + upsampled_obstacle_1_files
random.shuffle(balanced_files)  # Shuffle the data to mix the classes
split_index = int(len(matched_files) * 0.8)  # 80% training, 20% validation split

# Proceed with creating your datasets and dataloaders
train_files, val_files = balanced_files[:split_index], balanced_files[split_index:]
train_dataset = ObstacleDetectionDataset(train_files)
val_dataset = ObstacleDetectionDataset(val_files)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


# Model, loss, and optimizer setup
model = ObstacleDetectorModel().to(DEVICE)
model_path = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/models/obsta_model_10.pth'
model = torch.load(model_path)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 10
for epoch in range(EPOCHS):
    # Training phase
    model.train()
    total_train_loss = 0
    for pgm_input, seg_input, labels in train_loader:
        pgm_input, seg_input, labels = pgm_input.to(DEVICE), seg_input.to(DEVICE), labels.to(DEVICE)
        labels = labels.unsqueeze(1)  # Ensure labels are of shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(pgm_input, seg_input)
        loss = criterion(outputs, labels)
        print(loss)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for pgm_input, seg_input, labels in val_loader:
            pgm_input, seg_input, labels = pgm_input.to(DEVICE), seg_input.to(DEVICE), labels.to(DEVICE)
            labels = labels.unsqueeze(1)  # Ensure labels are of shape (batch_size, 1)
            outputs = model(pgm_input, seg_input)
            loss = criterion(outputs, labels)
            print("val: ",loss)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Validation Loss: {avg_val_loss:.4f}")

    # Save model and optimizer states
    save_path = f'/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/models/obsta_model_{10+epoch+1}.pth'
    torch.save(model, save_path)

PGM Image Batch Shape: torch.Size([32, 1, 180, 320])
Segmentation Image Batch Shape: torch.Size([32, 3, 180, 320])
Labels Batch Shape: torch.Size([32])


<ipython-input-9-4c5a633fe27e>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


tensor(0.0002, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0790, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(9.0893e-06, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(1.9272e-05, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0027, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0009, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0086, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(4.0226e-05, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(1.2159e-05, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(6.8693e-05, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0140, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(6.1630e-05, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<BinaryCrossE